## Introduction:
You can use this ipython notebook as a template for the rest of the homework.

### 0. Basic Useful Setups:

In [61]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

### 1. Load datasets

In [62]:
transform = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.MNIST(root='./data', train=True, 
                                      transform=transform, download=True)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                    transform=transform, download=True)

def getLoaders(batch_size=64, num_workers=4, val_size=0.2, random_seed=42):
    num_train = len(trainset)
    indices = list(range(num_train))
    split = int(np.floor(val_size * num_train))
    np.random.seed(random_seed)
    np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = torch.utils.data.sampler.SubsetRandomSampler(train_idx)
    valid_sampler = torch.utils.data.sampler.SubsetRandomSampler(valid_idx)
    
    trainloader = torch.utils.data.DataLoader(dataset=trainset, batch_size=batch_size, 
                                              num_workers=num_workers, sampler=train_sampler)
    valloader = torch.utils.data.DataLoader(dataset=trainset, batch_size=batch_size,
                                            num_workers=num_workers, sampler=valid_sampler)
    testloader = torch.utils.data.DataLoader(dataset=testset, batch_size=batch_size,
                                             num_workers=num_workers)
    return trainloader, valloader, testloader

### 1. LetNet Model

In [63]:
class LeNet(nn.Module):
    def __init__(self, opt='adam'):
        super(LeNet, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, 
        #                 padding=0, dilation=1, groups=1, bias=True)
        # torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1,
        #                    return_indices=False, ceil_mode=False)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, 
                               kernel_size=(5, 5), padding=(2, 2))
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, 
                               kernel_size=(5, 5), padding=(2, 2))
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.fc1 = nn.Linear(in_features=2450, out_features=500)
        self.fc2 = nn.Linear(in_features=500, out_features=10)
        
        # Optimizer
        if opt=='adam':
            self.optimizer = optim.Adam(self.parameters())
        elif opt=='rmsp':
            self.optimizer = optim.RMSprop(self.parameters())
        
        # Loss function
        self.criterion = nn.CrossEntropyLoss()
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.maxpool1(x)
        x = F.relu(self.conv2(x))
        x = self.maxpool2(x)
        x = x.view(-1, 50 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x),dim=1)
        return x
    
    def optimize(self, inputs, labels):
        self.optimizer.zero_grad()
        outputs = self(inputs)
        loss = self.criterion(outputs, labels)
        loss.backward()
        self.optimizer.step()
        return loss.data[0]
    
    def getLoss(self, inputs, labels):
        outputs = self(inputs)
        loss = self.criterion(outputs, labels)
        return loss.data[0]
    
    def getCorrect(self, inputs, labels):
        outputs = self(inputs)
        _, predicted = torch.max(outputs.data, 1)
        return (predicted == labels.data).sum()

### 2. Train and Evaluate LeNet Model

In [64]:
epochs = 5
lenet = LeNet('rmsp')
trainloader, valloader, testloader = getLoaders()

def getVariable(data):
    inputs, labels = data
    return Variable(inputs), Variable(labels)

for epoch in range(epochs):
    train_loss = 0.0
    train_correct = 0
    n_train = 0
    for i, data in enumerate(trainloader):
        inputs, labels = getVariable(data)
        train_loss += lenet.optimize(inputs, labels)
        train_correct += lenet.getCorrect(inputs, labels)
        n_train += len(labels)
    
    val_loss = 0.0
    val_correct = 0
    n_val = 0
    for i, data in enumerate(valloader):
        inputs, labels = getVariable(data)
        val_loss += lenet.getLoss(inputs, labels)
        val_correct += lenet.getCorrect(inputs, labels)
        n_val += len(labels)
        
    print('Epoch : ',epoch+1)
    print('Train Loss : ',train_loss/n_train)
    print('Val Loss : ',val_loss/n_val)
    print('Train Accuracy : ',1.0*train_correct/n_train)
    print('Val Accuracy : ',1.0*val_correct/n_val)
    print('#############################################')
    
test_loss = 0.0
test_correct = 0
n_test = 0
for i, data in enumerate(testloader):
    inputs, labels = getVariable(data)
    test_loss += lenet.getLoss(inputs, labels)
    test_correct += lenet.getCorrect(inputs, labels)
    n_test += len(labels)

print('Test Loss : ',test_loss/n_test)
print('Test Accuracy : ',1.0*test_correct/n_test)

('Epoch : ', 1)
('Train Loss : ', 0.03689910691976547)
('Val Loss : ', 0.0370267805258433)
('Train Accuracy : ', 0.09966666666666667)
('Val Accuracy : ', 0.09783333333333333)
#############################################
('Epoch : ', 2)
('Train Loss : ', 0.036898190905650455)
('Val Loss : ', 0.0370267805258433)
('Train Accuracy : ', 0.09966666666666667)
('Val Accuracy : ', 0.09783333333333333)
#############################################
('Epoch : ', 3)
('Train Loss : ', 0.036898190905650455)
('Val Loss : ', 0.0370267805258433)
('Train Accuracy : ', 0.09966666666666667)
('Val Accuracy : ', 0.09783333333333333)
#############################################
('Epoch : ', 4)
('Train Loss : ', 0.036898190905650455)
('Val Loss : ', 0.037022874275843304)
('Train Accuracy : ', 0.09966666666666667)
('Val Accuracy : ', 0.09783333333333333)
#############################################
('Epoch : ', 5)
('Train Loss : ', 0.036898190905650455)
('Val Loss : ', 0.03702808260917664)
('Train Accuracy :

In [ ]:
import sklearn